*Downloading and cleaning data on flights and airports*

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import urllib.request as req

# Download data on flights and airports

### Flights

Data comes from http://stat-computing.org/dataexpo/2009/the-data.html. Flights data relates to the year 2007 as it's the last year with complete information.

In [3]:
# Flights
filename = '2007.csv.bz2'
url = 'http://stat-computing.org/dataexpo/2009/'+str(filename)
%time req.urlretrieve(url,filename)
%time Flights = pd.read_csv(filename, compression='bz2')

# Save un-cleaned to pickle
Flights.to_pickle("Flights.pkl") # too big for Github, as >100 MB

print(Flights.shape)
print(Flights.columns)

Wall time: 38.5 s
Wall time: 44.3 s
(7453215, 29)
Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')


***Unique journeys for price scraping***

### Airports

Data comes from https://openflights.org/data.html

In [4]:
scrape_data = Flights.copy()
scrape_data["Journey"] = scrape_data["Origin"]+scrape_data["Dest"]
scrape_data = scrape_data.drop_duplicates(subset = "Journey", keep = "first")
scrape_data.head()
# scrape_data.to_pickle("scrape_data.pkl")

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Journey
0,2007,1,1,1,1232.0,1225,1341.0,1340,WN,2891,...,11,0,NaN,0,0,0,0,0,0,SMFONT
1,2007,1,1,1,1918.0,1905,2043.0,2035,WN,462,...,6,0,NaN,0,0,0,0,0,0,SMFPDX
6,2007,1,1,1,1936.0,1840,2217.0,2130,WN,409,...,7,0,NaN,0,46,0,0,0,1,SMFPHX
12,2007,1,1,1,2128.0,2035,2245.0,2200,WN,139,...,8,0,NaN,0,0,0,3,0,42,SMFSAN
25,2007,1,1,1,2119.0,2100,2310.0,2250,WN,961,...,8,0,NaN,0,19,0,1,0,0,SMFSEA


In [2]:
# Airports
filename = 'airports.csv'
url = 'http://stat-computing.org/dataexpo/2009/'+str(filename)
req.urlretrieve(url,filename)
Airports = pd.read_csv(filename)

# Save to pickle
Airports.to_pickle("Airports.pkl")

print('Shape:', Airports.shape)
Airports.head(2)

Shape: (3376, 7)


,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928


# Data cleaning for flights data

### Count number of flights between airports

In [5]:
# Select variables from which to create network. Must contain two columns of nodes, since 
# nx.from_pandas_edgelist() views each row as an edge. Can also contains attributes, e.g. Distance. 
FlightsNx = Flights[["Origin","Dest","Distance"]]

# Take counts (i.e. how many origin to destination flights in the dataset)
counts = pd.DataFrame(Flights.groupby(["Origin","Dest"])["Year"].count())
counts.columns = ["count"]
FlightsNx = FlightsNx.merge(counts, how = 'left', on = ["Origin","Dest"])

# Drop duplicates 
FlightsNx = FlightsNx.drop_duplicates()

# Count number of take-offs from origin: 
orig_takeoff = pd.DataFrame(FlightsNx.groupby(["Origin"])["count"].sum())
orig_takeoff.columns = ["origin_takeoff"]
# Merge to network dataset: 
FlightsNx = FlightsNx.merge(orig_takeoff, how = "left", on = "Origin")

# Count number of take-offs from dest
dest_takeoff = pd.DataFrame(FlightsNx.groupby(["Origin"])["count"].sum())
dest_takeoff.columns = ["dest_takeoff"]
FlightsNx = FlightsNx.merge(dest_takeoff, how = "left", left_on = "Dest", right_on = "Origin")

# Count number of landings at destination
dest_lands = pd.DataFrame(FlightsNx.groupby(["Dest"])["count"].sum())
dest_lands.columns = ["dest_landing"]
FlightsNx = FlightsNx.merge(dest_lands, how = "left", on = "Dest")

orig_lands = pd.DataFrame(FlightsNx.groupby(["Dest"])["count"].sum())
orig_lands.columns = ["orig_landing"]
FlightsNx = FlightsNx.merge(orig_lands, how = "left", left_on = "Origin", right_on = "Dest")

# Construct totals (flights)
FlightsNx["Origin_flights"] = FlightsNx["origin_takeoff"]+FlightsNx["orig_landing"]
FlightsNx["Destination_flights"] = FlightsNx["dest_takeoff"]+FlightsNx["dest_landing"]

# Drop auxiliary vars
FlightsNx.drop(["origin_takeoff","dest_takeoff","dest_landing","orig_landing"], axis = 1, inplace = True)

FlightsNx.head(2)

,Origin,Dest,Distance,count,Origin_flights,Destination_flights
0,SMF,ONT,389,4093,116158,83287.0
1,SMF,PDX,479,2721,116158,117237.0


### Implement as network
For function documentation, see:  
https://networkx.github.io/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html?highlight=from_pandas_edgelist#networkx.convert_matrix.from_pandas_edgelist

In [6]:
Network = nx.from_pandas_edgelist(FlightsNx, source = "Origin", target = "Dest", edge_attr = True)

### Degree distribution

In [7]:
# docs: https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.degree_centrality.html#networkx.algorithms.centrality.degree_centrality 
# Find degree: 
degree_dict = nx.algorithms.centrality.degree_centrality(Network) 
degree_values = list(degree_dict.values())
# Create dataframe
degrees = pd.DataFrame(degree_values)
degrees.columns = ["degree"]
degrees["degree_unnormalized"] = degrees["degree"]*(len(Network.nodes)-1)
degrees["airport"] = degree_dict.keys()

# Save to pickle for descriptive
degrees.to_pickle("Degrees.pkl")

In [8]:
# Add degree to FlightsNx data
FlightsNx = FlightsNx.merge(degrees, how = "left", left_on = "Origin", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = {"degree": "origin_degree", "degree_unnormalized": "origin_degree_unn"})

FlightsNx = FlightsNx.merge(degrees, how = "left", left_on = "Dest", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = {"degree": "dest_degree", "degree_unnormalized": "dest_degree_unn"})

FlightsNx.head(2)

,Origin,Dest,Distance,count,Origin_flights,Destination_flights,origin_degree,origin_degree_unn,dest_degree,dest_degree_unn
0,SMF,ONT,389,4093,116158,83287.0,0.142395,44.0,0.129450,40.0
1,SMF,PDX,479,2721,116158,117237.0,0.142395,44.0,0.139159,43.0


In [9]:
# Save cleaned to pickle
FlightsNx.to_pickle("FlightsNx.pkl") # too big for Github, as >100 MB